In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import numpy as np
import matplotlib.pyplot as plt
import os

BATCH_SIZE = 2  # Reducido para dataset pequeño
IMAGE_SIZE = (224, 224)  # Tamaño estándar para transfer learning
EPOCHS = 50  # Más épocas pero con early stopping
DATASET_DIR = "your_dataset" #Se encuentran las carpetas que contienen las imagenes para entrenar el modelo. En esta caso positive and negative

# Cargar dataset con data augmentation
def create_augmented_dataset():
    # Data augmentation para amplificar el dataset
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomBrightness(0.2),
        layers.RandomContrast(0.2),
    ])
    
    # Dataset original
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        DATASET_DIR,
        image_size=IMAGE_SIZE,
        batch_size=1,
        shuffle=True,
        seed=42
    )
    
    # Aplicar augmentation
    augmented_dataset = dataset.map(
        lambda x, y: (data_augmentation(x, training=True), y),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    # Combinar dataset original + augmentado
    combined_dataset = dataset.concatenate(augmented_dataset)
    combined_dataset = combined_dataset.concatenate(augmented_dataset)  # Triplicar datos
    
    return combined_dataset

class_names = ['negative', 'positive']
print("Clases:", class_names)

# Crear dataset aumentado
dataset = create_augmented_dataset()

# Función mejorada para desempaquetar
def unpack_and_convert(dataset):
    images = []
    labels = []
    
    for batch in dataset:
        batch_images, batch_labels = batch
        for i in range(len(batch_images)):
            # Normalizar las imágenes a [0,1]
            image = tf.cast(batch_images[i], tf.float32) / 255.0
            label = tf.cast(batch_labels[i], tf.int32)
            
            images.append(image)
            labels.append(label)
    
    return images, labels

# Aplicar la función
images, labels = unpack_and_convert(dataset)
total = len(images)
print(f"Total de ejemplos (con augmentation): {total}")

# Dividir los datos (70/15/15 para dataset pequeño)
train_size = int(0.7 * total)
val_size = int(0.15 * total)

# Mezclar los datos antes de dividir
indices = np.random.permutation(total)
images = [images[i] for i in indices]
labels = [labels[i] for i in indices]

train_images = images[:train_size]
train_labels = labels[:train_size]
val_images = images[train_size:train_size + val_size]
val_labels = labels[train_size:train_size + val_size]
test_images = images[train_size + val_size:]
test_labels = labels[train_size + val_size:]

print(f"Entrenamiento: {len(train_images)} ejemplos")
print(f"Validación: {len(val_images)} ejemplos")
print(f"Prueba: {len(test_images)} ejemplos")

# Crear datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# Configurar datasets con más augmentation en entrenamiento
def augment_train_data(image, label):
    # Augmentation adicional solo para entrenamiento
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_saturation(image, 0.7, 1.3)
    image = tf.image.random_hue(image, 0.1)
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.map(augment_train_data, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.shuffle(200).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

# Modelo con Transfer Learning (más robusto para pocos datos)
def create_transfer_model():
    # Cargar modelo preentrenado
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Congelar las primeras capas
    base_model.trainable = False
    
    # Agregar capas personalizadas
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

# Crear modelo
model = create_transfer_model()

# Compilar con learning rate más bajo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Callbacks para evitar overfitting
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7
    )
]

# Entrenar el modelo
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=callbacks,
    verbose=1
)

# Evaluar en conjunto de prueba
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=0)
print(f"Precisión en prueba: {test_accuracy:.4f}")

# Función de predicción mejorada
def predict_snake_improved(image_path, show_confidence_details=True):
    try:
        # Cargar y preprocesar la imagen
        img = tf.keras.utils.load_img(image_path, target_size=(224, 224))
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.cast(img_array, tf.float32) / 255.0
        img_array = tf.expand_dims(img_array, 0)
        
        # Hacer predicción
        prediction = model.predict(img_array, verbose=0)[0][0]
        
        if show_confidence_details:
            print(f"Valor raw de predicción: {prediction:.4f}")
            print(f"Umbral de decisión: 0.5")
        
        # Interpretar resultado con umbral ajustable
        threshold = 0.5
        if prediction > threshold:
            confidence = prediction
            result = f"Morelia viridis (confidence: {confidence:.1%})"
        else:
            confidence = 1 - prediction
            result = f"NOT Morelia viridis (confidence: {confidence:.1%})"
        
        return result, prediction
        
    except Exception as e:
        return f"Error: {e}", None

# Función para evaluar múltiples imágenes y ajustar umbral
def evaluate_predictions(image_paths, true_labels):
    """
    Evalúa múltiples imágenes para encontrar el mejor umbral
    image_paths: lista de rutas de imágenes
    true_labels: lista de etiquetas verdaderas (1 para serpiente, 0 para no serpiente)
    """
    predictions = []
    
    for path in image_paths:
        try:
            img = tf.keras.utils.load_img(path, target_size=(224, 224))
            img_array = tf.keras.utils.img_to_array(img)
            img_array = tf.cast(img_array, tf.float32) / 255.0
            img_array = tf.expand_dims(img_array, 0)
            pred = model.predict(img_array, verbose=0)[0][0]
            predictions.append(pred)
        except:
            predictions.append(0.5)  # Valor neutro si hay error
    
    # Probar diferentes umbrales
    best_threshold = 0.5
    best_accuracy = 0
    
    for threshold in np.arange(0.1, 0.9, 0.05):
        pred_labels = [1 if p > threshold else 0 for p in predictions]
        accuracy = sum(p == t for p, t in zip(pred_labels, true_labels)) / len(true_labels)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold
    
    print(f"Mejor umbral encontrado: {best_threshold:.2f}")
    print(f"Precisión con este umbral: {best_accuracy:.2%}")
    
    return best_threshold, predictions

# Graficar resultados
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Precisión de entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión de validación')
plt.title('Precisión del modelo')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Pérdida del modelo')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Usar la función mejorada
image_path = "target_image"
resultado, pred_value = predict_snake_improved(image_path)
print(resultado)

